# Model 3: Transformer Model

In [ ]:
from pathlib import Path
import numpy as np
import torch 

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

In [ ]:
# Import local modules from 'src/utils' as package 'utils'
import sys; sys.path.insert(0, '../')

In [ ]:
import utils

## Read Datasets from .csv

In [ ]:
from utils.file_io import read_trajectory_datasets

In [ ]:
data_folder = Path("../../data/")
train_set, test_set, validation_set, visualization_set = read_trajectory_datasets(data_folder, 0.85, 0.10, 0.045, 0.005, 256, normalize_features=True)

In [ ]:
# FIXME: Total loaded size correct?
input_shape, output_shape = 8, 3
print(f"Data shape {input_shape} / {output_shape} of total {len(train_set) + len(test_set) + len(validation_set) + len(visualization_set)} data rows!")

## Defining the transformer model

In [ ]:
import math
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer

### Partly PositionalEncoder

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 1024) -> None:
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x += self.pe[:x.size(0)]
        return self.dropout(x)

### Defining Transformer (Encoder/Decoder)

In [ ]:
class Transformer(nn.Module):
    def __init__(self, num_heads: int, model_dim: int, feedforward_hidden_dim: int, output_dim: int,
                 num_encoder_layers: int = 6, num_decoder_layers: int = 6, transformer_dropout: float = 0.1, pos_encoder_dropout: float = 0.25):
        super().__init__()
        self.model_type = 'Transformer'
        self.total_epochs = 0
        self.transformer = nn.Transformer(d_model=model_dim, nhead=num_heads, dim_feedforward=feedforward_hidden_dim, num_encoder_layers=num_encoder_layers, num_decoder_layers=num_decoder_layers, dropout=transformer_dropout)
        self.pos_encoder = PositionalEncoding(model_dim, pos_encoder_dropout)
        self.head = nn.Linear(model_dim, output_dim)

    def forward(self, source: Tensor, target: Tensor = None):
        # expect input shape to be (S, N, E) with S being the sequence length, N batch size and, E the input dimensionality
        # target_mask masks out all values right of the diagonal such that information from the target sequence cant bleed into the left hand side at training time
        target_mask = nn.Transformer.generate_square_subsequent_mask(source.shape[0], device=device)
        source = self.pos_encoder(source)
        return self.forward_train(source, target, target_mask) if target is not None else self.forward_inference(source, target_mask)
        
    def forward_train(self, source: Tensor, target: Tensor, target_mask: Tensor) -> Tensor:
        self.train()
        target = self.pos_encoder(target)
        target = self.transformer(source, target, tgt_mask=target_mask)
        return self.head(target)

    def forward_inference(self, source: Tensor, target_mask: Tensor) -> Tensor:
        self.eval()
        with torch.no_grad():
            target_pos_encoding = self.pos_encoder(torch.zeros_like(source, device=device))
            target = torch.zeros(source.shape[0] + 1, source.shape[1], source.shape[2], device=device)
            target[0, :, :] = torch.ones(source.shape[1:])
            for i, (source_batch, target_batch) in enumerate(zip(source, target)):
                out_target = self.transformer(source_batch, target_batch) 
                target[i + 1, :, :] = out_target + target_pos_encoding[i, :, :]
            return self.head(target[1:, :, :])    

### Defining Transformer encode only

In [ ]:
class TransformerEncoderOnly(nn.Module):
    def __init__(self, num_heads: int, model_dim: int, feedforward_hidden_dim: int, output_dim: int,
                 num_encoder_layers: int = 6, transformer_dropout: float = 0.1, pos_encoder_dropout: float = 0.25) -> None:
        super().__init__()
        self.model_type = 'Transformer'
        self.total_epochs = 0
        encoder_layers = TransformerEncoderLayer(model_dim, num_heads, feedforward_hidden_dim, transformer_dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, num_encoder_layers)   
        self.pos_encoder = PositionalEncoding(model_dim, pos_encoder_dropout)
        self.head = nn.Linear(model_dim, output_dim)

    def forward(self, source: Tensor, source_msk: Tensor = None) -> Tensor:
        # expect input shape to be (S, N, E) with S being the sequence length, N batch size and, E the input dimensionality
        # target_mask masks out all values right of the diagonal such that information from the target sequence cant bleed into the left hand side at training time
        source = self.pos_encoder(source)
        source = self.transformer_encoder(source, source_msk)
        return self.head(source)

In [ ]:
model = TransformerEncoderOnly(input_shape, 8, 32, output_shape)

## Load parameter, functions and dataloader

In [ ]:
import os

from torch.utils.data import DataLoader
from dotenv import load_dotenv

from utils.file_io import save_model
from utils.file_io import define_dataloader_from_subset
from utils.evaluation import compute_loss_on
from utils.optimizer import rate

In [ ]:
model_path = Path("../../models/transformer/").absolute()

In [ ]:
dotenv_path = model_path / ".env"
load_dotenv(dotenv_path=dotenv_path)

learning_rate = float(os.getenv("LEARNING_RATE"))
batch_size = int(os.getenv("BATCH_SIZE"))
num_epochs = int(os.getenv("NUM_EPOCHS"))

In [ ]:
def get_optimizer_function(model: nn.Module, learning_rate: float) -> torch.optim:
    return torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
def get_loss_function() -> nn.Module:
    return torch.nn.MSELoss()

In [ ]:
train_dataloader, validation_dataloader, test_dataloader = define_dataloader_from_subset(train_set, validation_set, test_set, batch_size=batch_size, shuffle=True)

## Define train methods

In [ ]:
from ray import train as ray_train
from ray.train import Checkpoint

In [ ]:
def train_epoch(train_dataloader: DataLoader, model, loss_function, optimizer,
                device: torch.device, report_interval: int = 1000):
    
    running_loss = 0
    last_loss = 0
    
    for i, (inputs, true_values) in enumerate(train_dataloader):
        
        inputs = inputs.to(device)
        true_values = true_values.to(device)
    
        inputs_shape, true_values_shape = inputs.size(), true_values.size()
        inputs = inputs.view(inputs_shape[1], inputs_shape[0], inputs_shape[2])
        true_values = true_values.view(true_values_shape[1], true_values_shape[0], true_values_shape[2])
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, true_values)
        running_loss += loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
    
        if i % report_interval == report_interval - 1:
            last_loss = running_loss / report_interval
            print(f"batch {i + 1}, Mean Squared Error: {last_loss}")
            running_loss = 0

    return last_loss

In [ ]:
def train(epochs: int, train_dataloader: DataLoader, validation_dataloader: DataLoader, model: nn.Module, 
          loss_function, optimizer, checkpoint_path: Path, device: torch.device = 'cpu', report_interval: int = 128, tune: bool = False) -> nn.Module:
    
    best_val_loss = float("inf")

    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    model.to(device)

    if tune:
        checkpoint = ray_train.get_checkpoint()

        if checkpoint:
            with checkpoint.as_directory() as checkpoint_dir:
                model_state = torch.load(os.path.join(checkpoint_dir, "checkpoint.pt"))
                model.load_state_dict(model_state)

    for epoch in range(model.total_epochs, epochs):
        print(f"Epoch: {epoch + 1}")

        model.train(True)
        avg_loss = train_epoch(train_dataloader, model, loss_function, optimizer, lr_scheduler, device, report_interval)
        model.eval()

        with torch.no_grad():
            avg_val_loss = compute_loss_on(validation_dataloader, model, loss_function, device=device)

        print(f"Loss on train: {avg_loss}, loss on validation: {avg_val_loss}")

        model.total_epochs += 1
    
        if avg_val_loss < best_val_loss or tune:
            best_val_loss = avg_val_loss            
            
            torch.save(model.state_dict(), checkpoint_path / "checkpoint.pt")

        if tune:
            ray_train.report(metrics={ "loss": float(avg_val_loss) }, checkpoint=Checkpoint.from_directory(checkpoint_path))

    return model

## Train the model with optuna hyperparameter tuning

In [ ]:
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.optuna import OptunaSearch
from typing import Dict
from utils.evaluation import compute_sliding_window_predictions, compute_losses_from

In [ ]:
def parameter_train(parameter: Dict, train_epochs: int, train_data: DataLoader, validation_data: DataLoader, model_input_shape: int,
                    model_output_shape: int, checkpoint_path: Path, device: torch.device) -> None:

    run_id = ray_train.get_context().get_trial_id()
    run_checkpoint = checkpoint_path / run_id
    run_checkpoint.mkdir(parents=True)

    model = TransformerEncoderOnly(model_input_shape, parameter["model_dim"], parameter["feedforward_dim"], model_output_shape,
                                   parameter["encoder_layer"], parameter["transformer_dropout"], parameter["pos_encoder_dropout"])

    optimizer = get_optimizer_function(model, parameter["lr"])
    loss_function = get_loss_function()

    _ = train(train_epochs, train_data, validation_data, model, loss_function, optimizer, run_checkpoint, device, report_interval=50, tune=True)

In [ ]:
learning_rate_radius = 1e-3
batch_size_radius = 10
num_samples = 100

In [ ]:
parameter_space = {
    "lr": tune.loguniform(learning_rate_radius - learning_rate, learning_rate + learning_rate_radius),
    "batch_size": tune.choice(list(range(batch_size - batch_size_radius, batch_size + batch_size_radius, 4))),
    "model_dim": tune.choice([8]),
    "feedforward_dim": tune.choice([32]),
    "encoder_layer": tune.choice([1, 2, 3]),
    "transformer_dropout": tune.uniform(0.1, 0.5),
    "pos_encoder_dropout": tune.uniform(0.1, 0.5)
}

In [ ]:
scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=num_epochs
)

In [ ]:
search_alg = OptunaSearch(
    metric="loss",
    mode="min"
) 

In [ ]:
if ray.is_initialized():
    ray.shutdown()

ray.init(runtime_env={ "py_modules": [utils] })

In [ ]:
ray_resources_manager = tune.with_resources(
    trainable=lambda param: parameter_train(param, num_epochs, train_dataloader, validation_dataloader, input_shape, output_shape, model_path, device),
    # See: https://stackoverflow.com/questions/58967793/what-is-the-way-to-make-tune-run-parallel-trials-across-multiple-gpus
    resources={ "cpu": 3, "gpu": 0.25 }
)

tuner = tune.Tuner(
    ray_resources_manager,
    param_space=parameter_space,
    tune_config=tune.TuneConfig(
        scheduler=scheduler,
        search_alg=search_alg,
         num_samples=num_samples
    )
)

In [ ]:
results = tuner.fit()

In [ ]:
if ray.is_initialized():
    ray.shutdown()

In [ ]:
# Save as csv file
results.get_dataframe().to_csv(model_path / "trail_grid.csv")

In [ ]:
best_result = results.get_best_result("loss", "min")
best_checkpoint = best_result.get_best_checkpoint("loss", "min")

best_model = torch.load(f"{best_checkpoint.path}/checkpoint.pt")

In [ ]:
print(f"Best trail by loss value {best_result.metrics['loss']}", "\n------")
for i in best_result.config:
    print(f"Best trail: {i} value {best_result.config[i]}")

## Evaluation

In [ ]:
from utils.visualization import create_trace_animation
from matplotlib import pyplot as plt
from IPython.display import HTML

In [ ]:
# Compute evaluation on the cpu
device = 'cpu'

In [ ]:
%matplotlib notebook
 
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  

### Loading the best model

In [ ]:
loss_function = get_loss_function()

In [ ]:
model = TransformerEncoderOnly(input_shape, best_result.config["model_dim"], best_result.config["feedforward_dim"], output_shape, best_result.config["encoder_layer"],
                                best_result.config["transformer_dropout"], best_result.config["pos_encoder_dropout"])
model.load_state_dict(best_model)
model.eval()

### Compute mean squared error

In [ ]:
y, y_true = compute_sliding_window_predictions(test_dataloader, model, device=device)
test_losses = compute_losses_from(y, y_true, loss_function)
print(f"The mean squared error on test is: {test_losses.mean()}")

### Draw prediction/truth traces

In [ ]:
%matplotlib notebook
 
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  

In [ ]:
visualization_dataloader = DataLoader(visualization_set, batch_size=batch_size, shuffle=False)
y, y_true = compute_sliding_window_predictions(visualization_dataloader, last_model, device=device)
y, y_true = y.numpy(), y_true.numpy()

In [ ]:
animation = create_trace_animation(y, y_true)
HTML(animation.to_jshtml())

In [ ]:
print(y_true)